In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saurabhbadole/bank-customer-churn-prediction-dataset")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\KUNAL\.cache\kagglehub\datasets\saurabhbadole\bank-customer-churn-prediction-dataset\versions\2


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle

In [2]:
# load the dataset
df = pd.read_csv("C:\\Users\\KUNAL\\.cache\\kagglehub\\datasets\\saurabhbadole\\bank-customer-churn-prediction-dataset\\versions\\2\\Churn_Modelling.csv")

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# preprocess the data
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [5]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
# Encode categorical variables
gen_encoder = LabelEncoder()
df["Gender"] = gen_encoder.fit_transform(df["Gender"])
df.head()

# df["Gender"] = df["Gender"].apply(lambda x: 1 if x == "Female" else 0)    ## just trying

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
geo_encoder = OneHotEncoder(sparse_output=False)
df_geo_encoded = geo_encoder.fit_transform(df[["Geography"]])

In [8]:
geography = pd.DataFrame(df_geo_encoded, columns=geo_encoder.get_feature_names_out(["Geography"]))

In [9]:
# combine the encoded geography with the original dataframe and drop the original Geography column
df = pd.concat([df, geography], axis=1)
df.drop(["Geography"], axis=1, inplace=True)

In [10]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [11]:
# saving the encoders
with open("gender_encoder.pkl","wb") as file:
    pickle.dump(gen_encoder, file)

with open("geo_encoder.pkl", "wb") as file:
    pickle.dump(geo_encoder, file)

In [12]:
X = df.drop("Exited", axis=1)
y = df["Exited"]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
with open("Scaler.pkl", "wb") as file:
    pickle.dump(sc, file)

In [16]:
# model training and saving